# MA 755 - Assignment 2

by Neha, Katrina, Chengdong

## **1. Get the data**
    1. Set up the working directory
    2. Download and unzip the data file
    3. Create a train set and a test set
  
## **2. Data discovery and visualisation**
    1. Scatter plot, box plot and histogram
    2. Examine the correlation

## **3. Prepare the data for Machine Learning algorithms**
    1. Collapsing the the levels of target variable
    2. Data cleaning - imputing missing values
  
## **4. Select and train different classification models**
    1. Logistic regression model 
    2. Random Forest model 
    3. SGD model 
    
## **5. Comparing the performance of the models**
    1. Measuring accuracy using cross-validation
    2. Confusion matrix
    3. Precision and recall
    4. Precision/Recall tradeoff
    5. ROC curve

## **6. Conclusion**
   

## 1. Get the data

### *1.1. Set up the working directory *

The first step is to create a workspace. We import the `os` module. Next we us the `getcwd` method to return a string representing the current working directory. Then we use the `chdir` method to change the current working directory to path mentioned. Finally we confirm if our working directory is set to the one specified by calling `getcwd` metod again.

In [34]:
import os
os.getcwd() 

'C:\\Users\\Chengdong Liang\\Desktop\\MA-755-Assignment-1-master'

The above output tell us that our current working directory is '/Users/akhoury/Desktop/MA 755'

###  *1.2. Download and unzip the data file*

We import the `os` and `zipfile` modules to fetch our data which is in the form of a zipped file. Next we import the `urllib` module that is used to fetch URLs. `COLLEGE_URL` represents the URL for our data. `COLLEGE_PATH` is the directory created in our working space to downlaod and save the dataset.

`fetch_college_data` function has two paramaeters : `college_url` for the url of our dataset and `college_path` for the path in our working space where the data should be downloaded and saved.`makedirs` method is used to create the `datatsets/college` directory and setting `exist_ok` to True tells Python to not raise an error if the directory already exists.`os.path.join` method combines the name of the directory where the data is stored and the name of the data file to create an abosulte path to the datafile.

In [35]:
import os
import zipfile
from six.moves import urllib
DOWNLOAD_ROOT = "https://ed-public-download.apps.cloud.gov/downloads/"
COLLEGE_PATH = "datasets/college"
COLLEGE_URL = DOWNLOAD_ROOT + "CollegeScorecard_Raw_Data.zip"

def fetch_college_data(college_url=COLLEGE_URL, college_path=COLLEGE_PATH):
    os.makedirs(college_path, exist_ok=True)
    zip_path = os.path.join(college_path, "Raw_Data.zip")
    urllib.request.urlretrieve(college_url, zip_path)
    college_zip = zipfile.ZipFile(zip_path)
    college_zip.extractall(path=college_path)
    college_zip.close()

Now we call the function `fetch_college_data`.

In [36]:
fetch_college_data()

It creates a `datasets/college` directory in the workspace, downloads the `CollegeScorecard_Raw_Data.zip` file and extracts all files from it in this directory.

The code below is the function to read in the csv file `MERGED2014_15_PP.csv` to a Pandas DataFrame.

In [50]:
import pandas as pd

DATA_PATH= "datasets/college/CollegeScorecard_Raw_Data"

def load_college_data(data_path=DATA_PATH):
    # `data_path` is relative to the directory containing this script
    csv_path = os.path.join(data_path, "MERGED2014_15_PP.csv")
    return pd.read_csv(csv_path)

### *1.3 Take a quick look at the data structure*

Next we call the function `load_college_data` which returns a Pandas DataFrame and assign it to `college_original`. We also confirm its type and then look at the first five rows of the DataFrame `college_original` using the `head()` method

In [65]:
college_original = load_college_data()
print(type(college_original))
college_original.head()

<class 'pandas.core.frame.DataFrame'>


C:\Users\Chengdong Liang\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2821: DtypeWarning: Columns (6,9,1379,1380,1381,1382,1383,1384,1385,1386,1387,1388,1389,1390,1391,1392,1393,1394,1395,1396,1397,1398,1399,1400,1401,1402,1403,1404,1405,1406,1407,1411,1425,1427,1503,1517,1529,1530,1532,1537,1540,1541,1542,1575,1576,1577,1578,1579,1580,1581,1582,1583,1584,1585,1586,1587,1588,1589,1590,1591,1592,1593,1594,1595,1596,1597,1598,1599,1600,1601,1602,1605,1606,1609,1610,1613,1614,1615,1725,1726,1727,1728,1729) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


,UNITID,OPEID,OPEID6,INSTNM,CITY,STABBR,ZIP,ACCREDAGENCY,INSTURL,NPCURL,...,D100_L4,TRANS_4,DTRANS_4,TRANS_L4,DTRANS_L4,ICLEVEL,UGDS_MEN,UGDS_WOMEN,CDR3_DENOM,CDR2_DENOM
0,100654,100200,1002,Alabama A & M University,Normal,AL,35762,Southern Association of Colleges and Schools C...,www.aamu.edu/,galileo.aamu.edu/netpricecalculator/npcalc.htm,...,NaN,0.0000,1044.0,NaN,NaN,1,0.4831,0.5169,1895.0,NaN
1,100663,105200,1052,University of Alabama at Birmingham,Birmingham,AL,35294-0110,Southern Association of Colleges and Schools C...,www.uab.edu,www.collegeportraits.org/AL/UAB/estimator/agree,...,NaN,0.2408,1246.0,NaN,NaN,1,0.4169,0.5831,3937.0,NaN
2,100690,2503400,25034,Amridge University,Montgomery,AL,36117-3553,Southern Association of Colleges and Schools C...,www.amridgeuniversity.edu,www2.amridgeuniversity.edu:9091/,...,NaN,0.0000,9.0,NaN,NaN,1,0.3986,0.6014,344.0,NaN
3,100706,105500,1055,University of Alabama in Huntsville,Huntsville,AL,35899,Southern Association of Colleges and Schools C...,www.uah.edu,finaid.uah.edu/,...,NaN,0.3477,765.0,NaN,NaN,1,0.5733,0.4267,1513.0,NaN
4,100724,100500,1005,Alabama State University,Montgomery,AL,36104-0271,Southern Association of Colleges and Schools C...,www.alasu.edu,www.alasu.edu/cost-aid/forms/calculator/index....,...,NaN,0.0000,1330.0,NaN,NaN,1,0.3877,0.6123,2137.0,NaN


The output tells us that `college_original` is a pandas Dataframe. Each row represents one college. There are 1743 attributes (columns). 

For further analysis, we need to subset this data set to include only our variables of interest.

### Subsetting the data

Based on the original dataset, a new dataframe is created, which includes the variables we mainly focus on:
* Dependent variable - 1
* Independent variables - 10

| **Variables** | **Description**      | **Type** | **Note**  |
| ------------- |:-------------:| -----------:|-----------:|
|CONTROL| Public or private institution |Categorical| Dependent|
|PCTFLOAN      | Rate of receiving federal loans | Numeric|Independent |
|PCTPELL      | Rate of receiving pell grant | Numeric|Independent |
| FEMALE      | Percentage of female students      | Numeric|  Independent |
| MARRIED | Percentage of married students|Numeric|   Independent |
| DEPENDENT |Percentage of married students     |Numeric|    Independent |
| VETERAN | Percentage of veteran students     |Numeric|   Independent |
|TUITFTE| Net tuition revenue per full-time student |Numeric| Independent|
|INEXPFTE|Instructional expenditures per full-time student |Numeric| Independent|
|DEP_INC_AVG|Average family income of dependent students |Numeric| Independent|
|IND_INC_AVG|Average family income of independent students |Numeric| Independent|

The subsetting of the DataFrame is done using square brackets and the names of the columns to be selected are placed inside the brackets. The selected columns are put in a new DataFrame called `college`.

Let's have a quick look at the first 5 rows of the DataFrame `college` by using the `head()` method.

In [66]:
college=college_original[['CONTROL','PCTFLOAN','PCTPELL','FEMALE','MARRIED', 'DEPENDENT', 'VETERAN',
                          'TUITFTE', 'INEXPFTE', 'DEP_INC_AVG', 'IND_INC_AVG' ]]
college.head()

,CONTROL,PCTFLOAN,PCTPELL,FEMALE,MARRIED,DEPENDENT,VETERAN,TUITFTE,INEXPFTE,DEP_INC_AVG,IND_INC_AVG
0,1,0.8284,0.7356,0.5262741999,0.0165942315,0.8704069538,PrivacySuppressed,9427.0,7437.0,33370.062501,9184.1313978
1,1,0.5214,0.3460,0.6323896753,0.1034554538,0.6883846794,0.0033305579,9899.0,17920.0,59212.341812,28786.1557
2,2,0.7795,0.6801,0.6465256798,0.2386706949,0.1057401813,PrivacySuppressed,12459.0,5532.0,30720.839981,18578.064211
3,1,0.4596,0.3072,0.4766449021,0.1320944249,0.6514314415,0.0090406831,8956.0,10211.0,67430.323934,25117.976718
4,1,0.7554,0.7347,0.6152291105,0.0097708895,0.8564690027,PrivacySuppressed,7733.0,7618.0,33576.943026,6974.2969861


From the first 5 rows of the data, we can observe that there is one variables (`CONTROL`) that has repetitive values, which indicates that it might be a categorical variable. 

Use the `value_counts()` method to have a closer look at this variable. This method gives us the frequency count for each of the unique values of the variable, arranged from highest to lowest.

In [67]:
college["CONTROL"].value_counts()

3    3703
1    2044
2    1956
Name: CONTROL, dtype: int64

The types of the institutions represented by these values are listed below
* 1= Public institutions
* 2= Private nonprofit institutions
* 3= Private for-profit institutions


So we can see that there are 3703 Private for-profit institutions, 2044 Public institutions and  1956 Private for-profit institutions.

The code below uses the `info()` method to provide a concise summary of the DataFrame `college`

In [68]:
college.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7703 entries, 0 to 7702
Data columns (total 11 columns):
CONTROL        7703 non-null int64
PCTFLOAN       6966 non-null float64
PCTPELL        6966 non-null float64
FEMALE         7497 non-null object
MARRIED        7497 non-null object
DEPENDENT      7497 non-null object
VETERAN        7497 non-null object
TUITFTE        7270 non-null float64
INEXPFTE       7270 non-null float64
DEP_INC_AVG    7497 non-null object
IND_INC_AVG    7497 non-null object
dtypes: float64(4), int64(1), object(6)
memory usage: 662.1+ KB


The output tells us that the DataFrame `college` has 6162 observations. The names of the variables, the number of non-null values and the type are displayed as well. 

For instance, the variable `PCTFLOAN` has 5594 non-null values and is of type `float`.

From the general information, as shown above, it can be seen that the variables `FEMALE`, `MARRIED`, `DEPENDENT`, `VETERAN`, `DEP_INC_AVG`, `IND_INC_AVG` are not numeric. And the variable `CONTROL` should be categorical.Therefore, we need to change the type of these variables. 

`to_numeric()` method takes a Pandas series as input and converts it to numeric, coercing when told to by setting `errors` to `coerce`. `as.type(category)` allows us to convert a column to a categorical variable.

In [69]:
college["FEMALE"]=pd.to_numeric(college["FEMALE"],errors='coerce')
college["MARRIED"]=pd.to_numeric(college["MARRIED"],errors='coerce')
college["DEPENDENT"]=pd.to_numeric(college["DEPENDENT"],errors='coerce')
college["VETERAN"]=pd.to_numeric(college["VETERAN"],errors='coerce')
college["DEP_INC_AVG"]=pd.to_numeric(college["DEP_INC_AVG"],errors='coerce')
college["IND_INC_AVG"]=pd.to_numeric(college["IND_INC_AVG"],errors='coerce')
college["CONTROL"] = college["CONTROL"].astype('category')

C:\Users\Chengdong Liang\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\Chengdong Liang\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\Chengdong Liang\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

Use the `info()` method to check whether the data type of these variables has been changed

In [70]:
college.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7703 entries, 0 to 7702
Data columns (total 11 columns):
CONTROL        7703 non-null category
PCTFLOAN       6966 non-null float64
PCTPELL        6966 non-null float64
FEMALE         6354 non-null float64
MARRIED        6540 non-null float64
DEPENDENT      6947 non-null float64
VETERAN        3295 non-null float64
TUITFTE        7270 non-null float64
INEXPFTE       7270 non-null float64
DEP_INC_AVG    6963 non-null float64
IND_INC_AVG    6963 non-null float64
dtypes: category(1), float64(10)
memory usage: 609.4 KB


From the output, all variables have been placed into proper data type.

### *1.3 Create a train set and a test set*

Split the dataset into a train set train_set and a test set test_set. 80% of the data will be used as the training set (named `train_set`), and 20% used as the test set (named `test_set`).

In [71]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(college, test_size=0.2, random_state=42)

This gives us our training set `train_set` and our test set `test_set`.

## 2. Data discovery and visualisation

Create a copy of the training set to start exploring.

In [72]:
college = train_set.copy()

### *2.1 Scatter plot, box plot and histogram*

## 3. Prepare the data for Machine Learning algorithms

We now create a new copy of the `train_set` and separate the target variable and independent variables. The DataFrame `college` contains the predictors and the series `college_type` consists of the variable `CONTROL`. We confirm their dimensions.

In [73]:
college = train_set.drop("CONTROL", axis=1)
college_type = train_set["CONTROL"].copy()
print(college.shape)
print(college_type.shape)

(6162, 10)
(6162,)


### *3.1 Collapsing the levels of target variable*

For our purpose, we need to train a binary classifier. Because our target variable has 3 levels, we need to collapse the number of levels into 2. Namely, we impute the target variable `CONTROL` into a new variable which only has two levels: 
* 1 = Public institutions
* 2 = Private institutions

In [116]:
college_type = college_type.replace({3:2})
college_type.value_counts()

2    4536
1    1626
3       0
Name: CONTROL, dtype: int64

### *3.2 Data cleaning - imputing missing values*

Most Machine Learning algorithms cannot work with missing features, so let's create a few functions to take care of them. We compute the median value on the training set and use it to fill the missing values in the training set.

In [75]:
from sklearn.preprocessing import Imputer

imputer = Imputer(strategy="median")

Use **fit()** method to add `imputer` instance to the dataframe `college` which contains our independent variables. <br>
The `Imputer` class helps to impute missing values, either using mean, median or the most frequent value of the row or column where missing values are located.

In [76]:
imputer.fit(college)

Imputer(axis=0, copy=True, missing_values='NaN', strategy='median', verbose=0)

Now, replace missing values by the computed medians in order to transform the training set:

In [83]:
X = imputer.transform(college)
type(X)

numpy.ndarray

Because the result is a Numpy array with transformed features. We can change the result to a Pandas DataFrame.

In [84]:
college_ind = pd.DataFrame(X, columns=college.columns)

In [87]:
college_ind.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6162 entries, 0 to 6161
Data columns (total 10 columns):
PCTFLOAN       6162 non-null float64
PCTPELL        6162 non-null float64
FEMALE         6162 non-null float64
MARRIED        6162 non-null float64
DEPENDENT      6162 non-null float64
VETERAN        6162 non-null float64
TUITFTE        6162 non-null float64
INEXPFTE       6162 non-null float64
DEP_INC_AVG    6162 non-null float64
IND_INC_AVG    6162 non-null float64
dtypes: float64(10)
memory usage: 481.5 KB
(6162, 10)
